In [3]:
    from sklearn.cluster import BisectingKMeans
import pickle
from dsipts import read_public_dataset, TimeSeries
data, columns = read_public_dataset(  dataset= 'weather', path= '/home/agobbi/Projects/ExpTS/data')
ts = TimeSeries('prova')
use_covariates = False
ts.load_signal(data, enrich_cat=  ['hour'],target_variables=['y'], past_variables=columns if use_covariates else [])
train,validation,test= ts.split_for_train(  perc_train= 0.8,  perc_valid= 0.1,  shift= 0,  skip_step=1,past_steps=16,future_steps=16)
token_split = 4
max_voc_size = 256
samples,length,_ = train.data['y'].shape
tmp = train.data['x_num_past'].squeeze().reshape(samples,-1,token_split)
_,sentence_length, _ = tmp.shape
tmp = tmp.reshape(-1,token_split)
with open('cluster_model.pkl','rb') as f:
    cl = pickle.load(f)


/home/agobbi/.conda/envs/tt/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
clusters = cl.predict(tmp)


In [50]:

samples = len(train.data['x_num_past'])
tmp=train.data['x_num_past'].squeeze().reshape(-1,token_split)
samples = len(test.data['y'])
y_test= cl.predict(test.data['y'].reshape(-1,token_split)).reshape(samples,token_split)

In [106]:
with open('res.pkl','rb') as f:
    real,predicted = pickle.load(f)

In [133]:
tot = []
for i in [1,2]:
    res = []
    data = tmp[np.where(clusters==i)[0]]
    for j in range(data.shape[1]):

        bootstrap_ci = bootstrap((data[:,j],), np.median,n_resamples=50, confidence_level=0.9,random_state=1, method='percentile')
        res.append([bootstrap_ci.confidence_interval.low,np.median(data[:,j]),bootstrap_ci.confidence_interval.high])
    tot.append(res)

In [110]:
from scipy.stats import bootstrap
tot_res = []
for j in range(len(y_test)):
    if j%100==0:
        print(j)
    res = []

    sample = predicted[j]
    for index in sample:
        data=tmp[np.where(clusters==index)[0]]

        for i in range(data.shape[1]):

            bootstrap_ci = bootstrap((data[:,i],), np.median,n_resamples=50, confidence_level=0.9,
                                     random_state=1, method='percentile')

            res.append([bootstrap_ci.confidence_interval.low,np.median(data[:,i]),bootstrap_ci.confidence_interval.high])
    tot_res.append(res)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200


In [111]:
tot_res = np.stack(tot_res)

In [113]:
%matplotlib qt
lag = 15
plt.plot(tot_res[:,lag,1])
plt.plot(test.data['y'][:,lag,0])

In [139]:
from dsipts import TimeSeries,read_public_dataset
from sklearn.cluster import BisectingKMeans
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from minigpt import GPT
from trainer import Trainer
import logging
logging.basicConfig(level=logging.INFO)
import argparse
from omegaconf import DictConfig, OmegaConf
from hydra.core.hydra_config import HydraConfig
import hydra
import pickle
import numpy as np
class SortDataset(Dataset):
    """ 
    Dataset for the Sort problem. E.g. for problem length 6:
    Input: 0 0 2 1 0 1 -> Output: 0 0 0 1 1 2
    Which will feed into the transformer concatenated as:
    input:  0 0 2 1 0 1 0 0 0 1 1
    output: I I I I I 0 0 0 1 1 2
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self,x,y,length, num_digits):
        self.length = length
        self.num_digits = num_digits
        self.x = torch.tensor(x).long()
        self.y = torch.tensor(y).long()
    
    def __len__(self):
        return len(self.x) # ...
    
    def get_vocab_size(self):
        return self.num_digits
    
    def get_block_size(self):
        return self.length * 2 - 1

    def __getitem__(self, idx):
        
        inp = self.x[idx]
        sol = self.y[idx]
        cat = torch.cat((inp, sol), dim=0)

        # the inputs to the transformer will be the offset sequence
        x = cat[:-1].clone()
        y = cat[1:].clone()
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:self.length-1] = -1
        return x, y
    
data, columns = read_public_dataset(  dataset= 'weather', path= '/home/agobbi/Projects/ExpTS/data')
ts = TimeSeries('prova')
use_covariates = False
ts.load_signal(data, enrich_cat=  ['hour'],target_variables=['y'], past_variables=columns if use_covariates else [])
train,validation,test= ts.split_for_train(  perc_train= 0.8,  perc_valid= 0.1,  shift= 0,  skip_step=1,past_steps=16,future_steps=16)
token_split = 4
max_voc_size = 256
samples,length,_ = train.data['y'].shape
tmp = train.data['x_num_past'].squeeze().reshape(samples,-1,token_split)
_,sentence_length, _ = tmp.shape
tmp = tmp.reshape(-1,token_split)
cl = BisectingKMeans(n_clusters=max_voc_size)
clusters = cl.fit_predict(tmp)

x_train = clusters.reshape(-1,sentence_length)
samples = train.data['y'].shape[0]
y_train = cl.predict(train.data['y'].squeeze().reshape(samples,-1,token_split).reshape(-1,token_split)).reshape(-1,sentence_length)
samples = validation.data['y'].shape[0]
y_validation = cl.predict(validation.data['y'].squeeze().reshape(samples,-1,token_split).reshape(-1,token_split)).reshape(-1,sentence_length)
x_validation = cl.predict(validation.data['x_num_past'].squeeze().reshape(samples,-1,token_split).reshape(-1,token_split)).reshape(-1,sentence_length)

samples = test.data['y'].shape[0]
y_test = cl.predict(test.data['y'].squeeze().reshape(samples,-1,token_split).reshape(-1,token_split)).reshape(-1,sentence_length)
x_test = cl.predict(test.data['x_num_past'].squeeze().reshape(samples,-1,token_split).reshape(-1,token_split)).reshape(-1,sentence_length)


train_dataset = SortDataset(x_train,y_train,sentence_length,max_voc_size)

/home/agobbi/.conda/envs/tt/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [141]:
len(train_dataset)

41979

In [142]:
next(iter(train_dataset))

(tensor([194, 163, 174, 176, 160, 168, 175]),
 tensor([ -1,  -1,  -1, 160, 168, 175, 160]))

In [144]:
train_loader = DataLoader(
    train_dataset,
    sampler=torch.utils.data.RandomSampler(train_dataset, replacement=True, num_samples=int(1e10)),
    shuffle=False,
    pin_memory=True,
    batch_size=16,
    num_workers=1,
)

In [149]:
next(iter(train_loader))[1].shape

torch.Size([16, 7])

In [151]:
torch.cat([encodings.argmax(dim=2),encodings.argmax(dim=2)[:,0:-1]]).shape


NameError: name 'encodings' is not defined